# **Image Classifier**

## Project Content <a id = 0></a>

### First Step: First Organization

1. [Introduction](#1)
2. [Loading libraries and packages to embark our new journey](#2)
3. [Checking The Dataset](#3)
4. [Creating a Data Pypeline and Loading The Dataset](#4)

### Second Step: Data Preprocessing

5. [Scaling The Data](#5)
6. [Seperating Data into Three Parts of Train, Validation and Test Sets](#6)
7. [Categorical Fields Analysis](#7)
8. [Feature Scaling](#8)
9. [Seperating Data into Two Parts of Train and Test](#9)

### Third and Final Step: Modeling

7. [Defining Modeling Functions](#7)
8. [Building The Model's Architecture](#8)
9. [First Time Training](#9)
10. [Plotting The Model's Loss and Accuracy](#10)
11. [Model's Performance Evaluation](#11)
12. [Testing The Model](#12)

***

# First Step: First Organization

***

## 1. Introduction <a id = 1></a>

This analysis is focused on the behavior of telecom customers who are more likely to terminate their contracts.

**Fields**
- Demographic information about customers | **Gender, Seniorcitizen, Partnere, Dependents**
- Main services that each customer has signed up for | **Phoneservice, Multiplelines, Internetservice**
- Internet-based services which customers have signed up for | **Onlinesecurity, Onlinebackup, Deviceprotection, Techsupport, and Streamingtv and streamingmovies**
- Customers' accounts information | **Tenure, Monthlycharges, Totalcharges**
- Customers' financial information | **Contract, Paymentmethod, Paperlessbilling**
- Customers who left within the last month | **Churn**

[Project Content](#0)

## 2. Loading libraries and packages to embark our new journey <a id = 2></a>

In [2]:
# Basic Python Packages
import os
import uuid

# Numpy Library
import numpy as np

# Pandas Library and Settings
import pandas as pd

# Visualization Libraries (Matplotlib, Seaborn)
import matplotlib.pyplot as plt

# Tensorflow Library
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices("GPU")
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
    
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, BatchNormalization, Dropout
from tensorflow.keras import callbacks
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.utils import plot_model
from tensorflow.keras.metrics import BinaryAccuracy, Recall, Precision
    
# OpenCV
import cv2
import imghdr

%matplotlib inline

Init Plugin
Init Graph Optimizer
Init Kernel


In [ ]:
conda install uuid

[Project Content](#0)

## 3. Checking The Dataset <a id = 3></a>

[Project Content](#0)

## 4. Creating a Data Pypeline and Loading The Dataset <a id = 4></a>

***

# Second Step: Data Preprocessing

***

## 5. Scaling The Data <a id = 5></a>

## 6. Seperating Data into Three Parts of Train, Validation and Test Sets <a id = 6></a>

***

# Third and Final Step: Modeling

***

## 7. Defining Modeling Functions <a id = 7></a>

## 8. Building The Model's Architecture <a id = 8></a>

## 9. First Time Training <a id = 9></a>

## 10. Plotting The Models's Loss and Accuracy <a id = 10></a>

## 11. Model's Performance Evaluation <a id = 12></a>

## 12. Testing The Model <a id = 12></a>